In [21]:
import pandas as pd

data_path = '../data/cat-in-the-dat-ii/'

train = pd.read_csv(data_path + 'train.csv', index_col= 'id')
test = pd.read_csv(data_path + 'test.csv', index_col= 'id')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col= 'id')

In [22]:
all_data = pd.concat([train, test])
all_data = all_data.drop('target', axis =  1) # 타깃값 제거
all_data

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,F,N,Red,Trapezoid,Hamster,Russia,Bassoon,...,0256c7a4b,02e7c8990,3.0,Contributor,Hot,c,U,Pw,6.0,3.0
1,1.0,1.0,0.0,F,Y,Red,Star,Axolotl,NaN,Theremin,...,52ead350c,f37df64af,3.0,Grandmaster,Warm,e,X,pE,7.0,7.0
2,0.0,1.0,0.0,F,N,Red,NaN,Hamster,Canada,Bassoon,...,745b909d1,NaN,3.0,NaN,Freezing,n,P,eN,5.0,9.0
3,NaN,0.0,0.0,F,N,Red,Circle,Hamster,Finland,Theremin,...,bdaa56dd1,f9d456e57,1.0,Novice,Lava Hot,a,C,NaN,3.0,3.0
4,0.0,NaN,0.0,T,N,Red,Triangle,Hamster,Costa Rica,NaN,...,NaN,c5361037c,3.0,Grandmaster,Cold,h,C,OZ,5.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,0.0,0.0,1.0,T,N,Red,Trapezoid,Axolotl,Costa Rica,Theremin,...,c1a4acfaf,45a68dd2e,2.0,Expert,Warm,n,R,dp,2.0,12.0
999996,0.0,0.0,0.0,F,N,Red,Polygon,Dog,Russia,Theremin,...,220190c9e,e48348d66,1.0,Contributor,NaN,n,X,US,1.0,5.0
999997,0.0,0.0,1.0,T,N,Red,Triangle,Hamster,NaN,Theremin,...,cc5495ab3,47021df0d,1.0,Grandmaster,Boiling Hot,m,P,TL,1.0,6.0


### BaseLine

In [17]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
all_data_encoded = encoder.fit_transform(all_data)

In [18]:
num_train = len(train)

X_train = all_data_encoded[:num_train]
X_test = all_data_encoded[num_train:]

y = train['target']



In [19]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y, test_size=0.1, stratify= y, random_state=10)

In [20]:
num_train = len(train)

X_train = all_data_encoded[:num_train]
X_test = all_data_encoded[num_train:]

y = train['target']



In [21]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y, test_size=0.1, stratify= y, random_state=10)

In [22]:
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression(max_iter = 1000, random_state = 42)
logistic_model.fit(X_train, y_train)

/Users/jhkim/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000, random_state=42)

In [23]:
y_valid_preds = logistic_model.predict_proba(X_valid)[:,1]

In [24]:
from sklearn.metrics import roc_auc_score

roc_auc = roc_auc_score(y_valid, y_valid_preds)
print(f"ROC AUC score: {roc_auc:.4f}")

ROC AUC score: 0.7859


In [25]:
y_preds = logistic_model.predict_proba(X_test)[:,1]
submission['target'] = y_preds
submission.to_csv('submission.csv')

### 결측치 처리 + 피쳐엔지니어링 + BaseLine

- 가설: 인공데이터이기 때문에 결측치가 고르게 배포되어 있을 것이다.
    => 놀랍게도 원래 데이터가 더 좋은 결과를 보였음...

In [150]:
import pandas as pd

data_path = '../data/cat-in-the-dat-ii/'

train = pd.read_csv(data_path + 'train.csv', index_col= 'id')
test = pd.read_csv(data_path + 'test.csv', index_col= 'id')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col= 'id')

In [151]:
train.head().T

id,0,1,2,3,4
bin_0,0.0,1.0,0.0,NaN,0.0
bin_1,0.0,1.0,1.0,0.0,NaN
bin_2,0.0,0.0,0.0,0.0,0.0
bin_3,F,F,F,F,T
bin_4,N,Y,N,N,N
nom_0,Red,Red,Red,Red,Red
nom_1,Trapezoid,Star,NaN,Circle,Triangle
nom_2,Hamster,Axolotl,Hamster,Hamster,Hamster
nom_3,Russia,NaN,Canada,Finland,Costa Rica
nom_4,Bassoon,Theremin,Bassoon,Theremin,NaN


In [152]:
all_data = pd.concat([train, test])
all_data = all_data.drop('target', axis =  1) # 타깃값 제거
all_data

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,F,N,Red,Trapezoid,Hamster,Russia,Bassoon,...,0256c7a4b,02e7c8990,3.0,Contributor,Hot,c,U,Pw,6.0,3.0
1,1.0,1.0,0.0,F,Y,Red,Star,Axolotl,NaN,Theremin,...,52ead350c,f37df64af,3.0,Grandmaster,Warm,e,X,pE,7.0,7.0
2,0.0,1.0,0.0,F,N,Red,NaN,Hamster,Canada,Bassoon,...,745b909d1,NaN,3.0,NaN,Freezing,n,P,eN,5.0,9.0
3,NaN,0.0,0.0,F,N,Red,Circle,Hamster,Finland,Theremin,...,bdaa56dd1,f9d456e57,1.0,Novice,Lava Hot,a,C,NaN,3.0,3.0
4,0.0,NaN,0.0,T,N,Red,Triangle,Hamster,Costa Rica,NaN,...,NaN,c5361037c,3.0,Grandmaster,Cold,h,C,OZ,5.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,0.0,0.0,1.0,T,N,Red,Trapezoid,Axolotl,Costa Rica,Theremin,...,c1a4acfaf,45a68dd2e,2.0,Expert,Warm,n,R,dp,2.0,12.0
999996,0.0,0.0,0.0,F,N,Red,Polygon,Dog,Russia,Theremin,...,220190c9e,e48348d66,1.0,Contributor,NaN,n,X,US,1.0,5.0
999997,0.0,0.0,1.0,T,N,Red,Triangle,Hamster,NaN,Theremin,...,cc5495ab3,47021df0d,1.0,Grandmaster,Boiling Hot,m,P,TL,1.0,6.0


#### Binary

In [153]:
# 이진피처 인코딩 - 손으로.

all_data['bin_3'] = all_data['bin_3'].map({'F':0, 'T':1})
all_data['bin_4'] = all_data['bin_4'].map({'N':0, 'Y':1})

In [154]:
binary_data = ['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4']
all_data[binary_data].isna().sum()

bin_0    29795
bin_1    30041
bin_2    29902
bin_3    29965
bin_4    29998
dtype: int64

In [155]:
for column in binary_data: # 중간값으로 대체
    all_data[column].fillna(0.5, inplace=True)

#### Nominal

In [156]:
nominal_data = ['nom_' + str(i) for i in range(10)]
all_data[nominal_data].isna().sum()

nom_0    30314
nom_1    30103
nom_2    30214
nom_3    30297
nom_4    30028
nom_5    29690
nom_6    30143
nom_7    30006
nom_8    29711
nom_9    30133
dtype: int64

In [157]:
for column in nominal_data: # "Missing"으로 표시
    all_data[column].fillna("Missing", inplace=True)

#### Ordinal

In [158]:
all_data['ord_0'] = all_data['ord_0'].fillna(2.0)

In [159]:
ordinal_data = ['ord_1', 'ord_2']
all_data[ordinal_data].isna().sum()

for column in ordinal_data: # Missing으로 처리
    all_data[column].fillna('Missing', inplace=True)

In [160]:
# ord_1, ord_2 인코딩: Missing이 중앙값으로 가도록 처리

ord1dict = {'Novice': 0, 'Contributor': 1, 'Expert': 2, 'Missing': 3, 'Master': 4, 'Grandmaster': 5}
ord2dict = {'Freezing': 0, 'Cold': 1, 'Warm': 2, 'Missing' : 3, 'Hot' : 4, 'Boiling Hot': 5, 'Lava Hot': 6}

all_data['ord_1'] = all_data['ord_1'].map(ord1dict)
all_data['ord_2'] = all_data['ord_2'].map(ord2dict)

In [161]:
ordinal_data2 = ['ord_3', 'ord_4', 'ord_5']
all_data[ordinal_data2].isna().sum()

ord_3    29969
ord_4    29863
ord_5    29760
dtype: int64

In [162]:
# ord_3, ord_4, ord_5 인코딩

from sklearn.preprocessing import OrdinalEncoder
import numpy as np

ord_345 = ['ord_3', 'ord_4', 'ord_5']

ord_encoder = OrdinalEncoder()

all_data[ord_345] = ord_encoder.fit_transform(all_data[ord_345])

# 값을 가운데로 넣음.
def map_categories_with_nan_middle(categories):
    categories = sorted([cat for cat in categories if cat is not np.nan])
    middle_index = len(categories) // 2
    category_mapping = {cat: idx for idx, cat in enumerate(categories)}
    return category_mapping, middle_index

for feature in ['ord_3', 'ord_4', 'ord_5']:
    categories = all_data[feature].unique()
    category_mapping, nan_index = map_categories_with_nan_middle(categories)
    all_data[feature] = all_data[feature].map(category_mapping)
    all_data[feature] = all_data[feature].fillna(nan_index)

In [163]:
day_data = ['day','month']
all_data[day_data].isna().sum()

for column in day_data: # 빈도가 가장 높은 값으로 대체
    all_data[column].fillna(all_data[column].median(), inplace=True)

In [164]:
all_data.isna().sum()

bin_0    0
bin_1    0
bin_2    0
bin_3    0
bin_4    0
nom_0    0
nom_1    0
nom_2    0
nom_3    0
nom_4    0
nom_5    0
nom_6    0
nom_7    0
nom_8    0
nom_9    0
ord_0    0
ord_1    0
ord_2    0
ord_3    0
ord_4    0
ord_5    0
day      0
month    0
dtype: int64

#### Feature Engineering

In [165]:
## Nominal Data

nom_features = ['nom_' + str(i) for i in range(10)]

from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder()
encoded_nom_matrix = onehot_encoder.fit_transform(all_data[nom_features])
all_data = all_data.drop(nom_features, axis = 1)
encoded_nom_matrix

<1000000x5437 sparse matrix of type '<class 'numpy.float64'>'
	with 10000000 stored elements in Compressed Sparse Row format>

In [166]:
date_features = ['day', 'month']

onehot_encoder = OneHotEncoder()
encoded_date_matrix = onehot_encoder.fit_transform(all_data[date_features])
all_data = all_data.drop(date_features, axis = 1)

encoded_date_matrix

<1000000x19 sparse matrix of type '<class 'numpy.float64'>'
	with 2000000 stored elements in Compressed Sparse Row format>

### 피쳐 정규화

In [167]:
from sklearn.preprocessing import MinMaxScaler

ord_features = ['ord_' + str(i) for i in range(6)]

all_data[ord_features] = MinMaxScaler().fit_transform(all_data[ord_features])

In [83]:
all_data.head().T

id,0,1,2,3,4
bin_0,0.000000,1.000000,0.000000,0.500000,0.00000
bin_1,0.000000,1.000000,1.000000,0.000000,0.50000
bin_2,0.000000,0.000000,0.000000,0.000000,0.00000
bin_3,0.000000,0.000000,0.000000,0.000000,1.00000
bin_4,0.000000,1.000000,0.000000,0.000000,0.00000
ord_0,1.000000,1.000000,1.000000,0.000000,1.00000
ord_1,0.250000,1.000000,0.000000,0.000000,1.00000
ord_2,0.600000,0.400000,0.000000,1.000000,0.20000
ord_3,0.142857,0.285714,0.928571,0.000000,0.50000
ord_4,0.800000,0.920000,0.600000,0.080000,0.08000


In [168]:
from scipy import sparse

all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data),
                               encoded_nom_matrix,
                               encoded_date_matrix],
                               format ='csr')

In [169]:
all_data_sprs.shape

(1000000, 5467)

#### Train, Test Split

In [170]:
num_train = len(train)

X_train = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

y = train['target']



In [171]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y, test_size=0.1, stratify= y, random_state=10)

In [172]:
%%time

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression()

lr_params = {'C': [0.1,0.125,0.2], 'max_iter':[800,900,1000],
'solver': ['liblinear'], 'random_state': [42]}

gridsearch_logistic_model = GridSearchCV(estimator=logistic_model, param_grid=lr_params, scoring='roc_auc', cv = 5)
gridsearch_logistic_model.fit(X_train, y_train)

print('최적의 파라미터는', gridsearch_logistic_model.best_params_)

최적의 파라미터는 {'C': 0.1, 'max_iter': 800, 'random_state': 42, 'solver': 'liblinear'}
CPU times: user 2min 57s, sys: 987 ms, total: 2min 58s
Wall time: 3min 1s


In [174]:
from sklearn.metrics import roc_auc_score

y_valid_preds = gridsearch_logistic_model.predict_proba(X_valid)[:, 1]

roc_auc = roc_auc_score(y_valid, y_valid_preds)

print(f"검증 데이터 ROC AUC: {roc_auc:.4f}")


검증 데이터 ROC AUC: 0.7792


In [175]:
y_preds = gridsearch_logistic_model.best_estimator_.predict_proba(X_test)[:,1]

submission['target'] = y_preds
submission.to_csv('submission.csv')

### 최종 제출

In [176]:
num_train = len(train)

X_train = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

y = train['target']

In [177]:
%%time

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression()

lr_params = {'C': [0.1,0.125,0.2], 'max_iter':[800,900,1000],
'solver': ['liblinear'], 'random_state': [42]}

gridsearch_logistic_model = GridSearchCV(estimator=logistic_model, param_grid=lr_params, scoring='roc_auc', cv = 5)
gridsearch_logistic_model.fit(X_train, y)

print('최적의 파라미터는', gridsearch_logistic_model.best_params_)

최적의 파라미터는 {'C': 0.1, 'max_iter': 800, 'random_state': 42, 'solver': 'liblinear'}
CPU times: user 3min 32s, sys: 1.48 s, total: 3min 33s
Wall time: 3min 37s


In [178]:
y_preds = gridsearch_logistic_model.best_estimator_.predict_proba(X_test)[:,1]

submission['target'] = y_preds
submission.to_csv('submission.csv')